## 4.2 손실 함수

신경망은 '하나의 특정한 지표'를 기준으로 최적의 매개변수 값을 탐색 <br>
이 지표가 신겸망에서는 '손실 함수(loss function)' 이라고 칭한다. <br>
손실 함수는 임의의 함수를 사용할 수도 있지만 일반적으로 오차제곱합과 교차 엔트로피 오차를 사용 <br>

손실 함수는 신경망 성능의 '나쁨'을 나타내는 지표<br>
현재 신경망이 훈련 데이터를 엄라나 잘 처리하지 '못'하느냐를 측정 <br>

## 4.2.1 오차제곱합

In [1]:
# 이미지가 해당 인덱스일 확률 의미. (ex. 이미지가 '1'일 확률은 0.05)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
# 정답 레이블 (one-hot encoding)
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0] 

In [2]:
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [4]:
import numpy as np
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [7]:
# '7'일 확률이 가장 높다고 추정 (0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

## 4.2.2 교차 엔트로피 오차

교차 엔트포리의 오차는 정답일 때의 출력이 전체 값을 정한다. <br>


In [9]:
def cross_entropy_error(y, t):
    delta = 1e-7
    # log0은 inf이므로, 아주 작은 값인 delta를 더한다.
    return -np.sum(t * np.log(y+delta))

In [10]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

In [12]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

## 4.2.3 미니배치 학습

In [15]:
import numpy as np
import pickle
from dataset.mnist import load_mnist

(X_train, y_train), (X_test, y_test) = load_mnist(one_hot_label=True, normalize=False)
print('X_train shape = ',X_train.shape)
print('y_train shape = ',y_train.shape)

X_train shape =  (60000, 784)
y_train shape =  (60000, 10)


In [19]:
# 무작위 10장 추출 (minibatch)
train_size = X_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = X_train[batch_mask]
y_batch = y_train[batch_mask]

## 4.2.4 (배치용) 교차 엔트로피 오차 구현하기

In [24]:
def cross_entrophy_error(y, t):
    # y가 1차원 즉, 데이터 하나라면 reshape 함수로 데이터 형상 변환.
    if y.ndin == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [36]:
# 정답 레이블이 원-핫 인코딩이 아닌 경우

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

## 4.2.5 왜 손실 함수를 설정하는가?

신경망 학습에서의 '미분'의 역할에 주목하자 <br>
신경망 학습에서는 최적의 매개변수(가중치와 편향)를 탐색 할 때 손실함수의 값을 가능한 작게 하는 매개변수 값을 찾는 것 <br>
매개변수의 미분(정확히는 기울기)을 계산하고, 그 미분 값을 단서로 매개변수의 값을 서서히 갱신하는 과정 반복 <br>
가중치 매개변수의 손실 함수의 미분이란 '가중치 매개변수의 값을 아주 조금 변화 시켰을 때, 손실 함수가 어떻게 변하나' 라는 의미 <br>
<br>
정확도를 지표로 삼아서는 안되는 이유 -> 대부분의 장소에서 미분값이 0이 될 시 매개변수를 갱신할 수 없음 

## 4.4.1 경사법(경사 하강법)

기계학습 문제 대부분은 학습 단계에서 최적의 매개변수를 찾아낸다 <br>
여기에서 최적이란 손실 함수가 최솟값이 될 때의 매개변수 값. <br>
그러나 주의할 점은 각 지점에서 함수의 값을 낮추는 방안을 제시하는 지표가 '기울기'라는 것 <br>
해당 기울기가 가리키는 곳에 정말 함수의 최솟값이 존재하는지 보장 X <br>

<br>
경사법은 현 위치에서 기울어진 방향으로 일정 거리만큼 이동 <br>
